In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import os
import random
import stellargraph as sg
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec, link_classification
from tensorflow import keras
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

data_dir = "data/"  # working directory
walk_length = 3
number_of_walks = 3
batch_size = 32
epochs = 50
layer_sizes = [16]
kfold = 10

# Read edge list
edgelist = pd.read_csv(os.path.join(data_dir, "pair_Sa"),
                       sep="\t", header=None, names=["source", "target", "label", "eo1", "eo2"])

# Read node list
file_nodes = pd.read_csv(os.path.join(data_dir, "pair_content_Sa"), sep="\t")
nodes = file_nodes.set_index('ID')
feats = nodes.columns[:-1]  # Exclude label column

# Create graph
G_all_nx = nx.from_pandas_edgelist(edgelist, edge_attr="label")
nx.set_node_attributes(G_all_nx, "label", "label")
G_all = sg.StellarGraph.from_networkx(G_all_nx, node_features=nodes[feats])

# Store probabilities for each method
synergy_probs = {"Hadamard": [], "L1-norm": [], "L2-norm": [], "Average": [], "Classification-based": []}
antagonistic_probs = {"Hadamard": [], "L1-norm": [], "L2-norm": [], "Average": [], "Classification-based": []}
lbl_p_synergistic, lbl_p_antagonistic = [], []

# Train Attri2Vec model
generator = Attri2VecNodeGenerator(G_all, batch_size)
attri2vec = Attri2Vec(layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None)
x_inp, x_out = attri2vec.in_out_tensors()
embedding_model = keras.Model(inputs=x_inp, outputs=x_out)

for k in range(kfold):
    print(f"Processing Fold {k+1}/{kfold}...")

    # Read train and test interaction data
    edge_t = pd.read_csv(os.path.join(data_dir, f"{k+1}_t"), sep="\t",
                         header=None, names=["source", "target", "label", "eo1", "eo2"])
    edge_p = pd.read_csv(os.path.join(data_dir, f"{k+1}_p"), sep="\t",
                         header=None, names=["source", "target", "label", "eo1", "eo2"])

    # Convert labels from [1,2,3] → [0,1,2] for classification
    edge_t["label"] = edge_t["label"] - 1
    edge_p["label"] = edge_p["label"] - 1

    # Generate embeddings
    node_ids = nodes.index
    node_gen = generator.flow(node_ids)
    emb = embedding_model.predict(node_gen, workers=4, verbose=1)

    # Function to extract edge features
    def extract_features(edge_data):
        feat_h, feat_L1, feat_L2, feat_av, feat_cmp_av = [], [], [], [], []
        for i in range(len(edge_data)):
            try:
                n1 = np.where(nodes.index == edge_data["source"][i])[0][0]
                n2 = np.where(nodes.index == edge_data["target"][i])[0][0]

                feat_h.append(np.ravel(emb[n1] * emb[n2]))  # Hadamard
                feat_L1.append(np.ravel(np.abs(emb[n1] - emb[n2])))  # L1-norm
                feat_L2.append(np.ravel((emb[n1] - emb[n2]) ** 2))  # L2-norm
                feat_av.append(np.ravel((emb[n1] + emb[n2]) / 2))  # Average
                feat_cmp_av.append((nodes.loc[edge_data["source"][i], feats] +
                                    nodes.loc[edge_data["target"][i], feats]) / 2)  # Classification-based
            except IndexError:
                print(f"Skipping invalid edge: {edge_data.iloc[i].to_dict()}")
        
        return np.array(feat_h), np.array(feat_L1), np.array(feat_L2), np.array(feat_av), np.array(feat_cmp_av)

    # Extract features for training and testing
    h_feat_t, L1_feat_t, L2_feat_t, av_feat_t, cmp_av_feat_t = extract_features(edge_t)
    h_feat_p, L1_feat_p, L2_feat_p, av_feat_p, cmp_av_feat_p = extract_features(edge_p)

    # Prepare labels
    lbl_p = edge_p["label"].values

    # **Separate Labels for Two Models**
    lbl_p_synergistic.extend((lbl_p == 1).astype(int))  # 1 for synergy, 0 for others
    lbl_p_antagonistic.extend((lbl_p == 2).astype(int))  # 1 for antagonistic, 0 for others

    # Function to train Random Forest for Binary Classification
    def train_rf(train_X, train_y, test_X):
        rf_model = RandomForestClassifier(n_estimators=100, random_state=seed)
        rf_model.fit(train_X, train_y)
        pred_proba = rf_model.predict_proba(test_X)[:, 1]  # Only positive class probability
        return pred_proba

    # **Train Separate Models for Each Feature Extraction Method**
    synergy_probs["Hadamard"].append(train_rf(h_feat_t, (edge_t["label"] == 1).astype(int), h_feat_p))
    synergy_probs["L1-norm"].append(train_rf(L1_feat_t, (edge_t["label"] == 1).astype(int), L1_feat_p))
    synergy_probs["L2-norm"].append(train_rf(L2_feat_t, (edge_t["label"] == 1).astype(int), L2_feat_p))
    synergy_probs["Average"].append(train_rf(av_feat_t, (edge_t["label"] == 1).astype(int), av_feat_p))
    synergy_probs["Classification-based"].append(train_rf(cmp_av_feat_t, (edge_t["label"] == 1).astype(int), cmp_av_feat_p))

    antagonistic_probs["Hadamard"].append(train_rf(h_feat_t, (edge_t["label"] == 2).astype(int), h_feat_p))
    antagonistic_probs["L1-norm"].append(train_rf(L1_feat_t, (edge_t["label"] == 2).astype(int), L1_feat_p))
    antagonistic_probs["L2-norm"].append(train_rf(L2_feat_t, (edge_t["label"] == 2).astype(int), L2_feat_p))
    antagonistic_probs["Average"].append(train_rf(av_feat_t, (edge_t["label"] == 2).astype(int), av_feat_p))
    antagonistic_probs["Classification-based"].append(train_rf(cmp_av_feat_t, (edge_t["label"] == 2).astype(int), cmp_av_feat_p))

# Compute AUC scores separately
print("\nAUC (Synergistic vs. Rest):")
for method in synergy_probs:
    print(f"{method}: {roc_auc_score(lbl_p_synergistic, np.hstack(synergy_probs[method])):.4f}")

print("\nAUC (Antagonistic vs. Rest):")
for method in antagonistic_probs:
    print(f"{method}: {roc_auc_score(lbl_p_antagonistic, np.hstack(antagonistic_probs[method])):.4f}")


Processing Fold 1/10...
4/4 [==============================] - 0s 34ms/step
Processing Fold 2/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 3/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 4/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 5/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 6/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 7/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 8/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 9/10...
4/4 [==============================] - 0s 35ms/step
Processing Fold 10/10...
4/4 [==============================] - 0s 35ms/step

AUC (Synergistic vs. Rest):
Hadamard: 0.5557
L1-norm: 0.5559
L2-norm: 0.5396
Average: 0.5644
Classification-based: 0.6282

AUC (Antagonistic vs. Rest):
Hadamard: 0.6198
L1-norm: 0.5775
L2-norm: 0.5946
Average: 0.6253
Classification-based: